In [1]:
import numpy as np
import pandas as pd

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [74]:
df=pd.read_csv('/content/train.xls')
df.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [75]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
df.head()

Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         0       3    male  22.0      1      0   7.2500        S
1         1       1  female  38.0      1      0  71.2833        C
2         1       3  female  26.0      0      0   7.9250        S
3         1       1  female  35.0      1      0  53.1000        S
4         0       3    male  35.0      0      0   8.0500        S

In [76]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)
X_train.head()

Pclass     Sex   Age  SibSp  Parch     Fare Embarked
331       1    male  45.5      0      0  28.5000        S
733       2    male  23.0      0      0  13.0000        S
382       3    male  32.0      0      0   7.9250        S
704       3    male  26.0      1      0   7.8542        S
813       3  female   6.0      4      2  31.2750        S

In [77]:
trf1=ColumnTransformer([('impute_age',SimpleImputer(),['Age']),('impute_embarked',SimpleImputer(strategy='most_frequent'),['Embarked'])],remainder='passthrough')

In [78]:
trf2=ColumnTransformer([('ohe_sex',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,3])],remainder='passthrough')

In [79]:
trf3=ColumnTransformer([('scale',MinMaxScaler(),slice(0,10))])

In [80]:
trf4=SelectKBest(score_func=chi2,k=5)

In [81]:
trf5=DecisionTreeClassifier()

In [82]:
pipe=Pipeline([('trf1',trf1),('trf2',trf2),('trf3',trf3),('trf4',trf4),('trf5',trf5)])

In [84]:
pipe.fit(X_train,y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  ['Age']),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['Embarked'])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 3])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x7c0f3c912ac0>)),
                ('trf5', DecisionTreeClassifier())])

In [85]:
from sklearn import set_config
set_config(display='diagram')

In [86]:
y_pred=pipe.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8044692737430168

In [87]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), ['Age']),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  ['Embarked'])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 3])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=5, score_func=<function chi2 at 0x7c0f3c912ac0>),
 'trf5': DecisionTreeClassifier()}

In [89]:
pipe.named_steps['trf1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), ['Age']),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 ['Embarked'])])

In [91]:
pipe.named_steps['trf1'].transformers_

[('impute_age', SimpleImputer(), ['Age']),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), ['Embarked']),
 ('remainder',
  FunctionTransformer(accept_sparse=True, check_inverse=False,
                      feature_names_out='one-to-one'),
  [0, 1, 3, 4, 5])]

In [92]:
pipe.named_steps['trf1'].transformers_[0][1]

SimpleImputer()

In [94]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

In [95]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()
#cross validation using pipeline

np.float64(0.789421845759874)

In [96]:
#gridsearch using pipeline
params={'trf5__max_depth':[1,2,3,4,5,None]}
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(X_train,y_train)
grid.best_score_

np.float64(0.8033093666896484)

In [97]:
grid.best_params_

{'trf5__max_depth': 3}